In [74]:
import pandas as pd
import numpy as np

In [75]:
df = pd.read_csv("cosmicclassifierTraining.csv")
df.sample(5)

,Atmospheric Density,Surface Temperature,Gravity,Water Content,Mineral Abundance,Orbital Period,Proximity to Star,Magnetic Field Strength,Radiation Levels,Atmospheric Composition Index,Prediction
15993,4.037125,-0.056487,5.023511,-1.008814,-1.228337,-0.346639,-0.435115,Category_12,Category_7,-0.107971,NaN
50389,4.134482,3.325535,0.196664,1.543636,1.258738,1.025305,-2.059074,Category_11,Category_9,0.797615,4.0
55622,3.618479,-0.243115,-0.177090,-0.222510,-0.450821,-0.793909,-0.524543,Category_10,Category_4,-0.320902,0.0
25582,-1.120491,-1.480039,0.652733,-0.661714,1.545302,0.662828,NaN,Category_11,Category_7,1.312761,8.0
22281,-0.110103,0.430885,3.236913,-3.648288,1.764368,1.262199,-0.400121,Category_7,Category_12,-0.988622,8.0


In [76]:
df.shape

(60000, 11)

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Atmospheric Density            57016 non-null  float64
 1   Surface Temperature            56968 non-null  float64
 2   Gravity                        57016 non-null  float64
 3   Water Content                  56923 non-null  float64
 4   Mineral Abundance              57079 non-null  float64
 5   Orbital Period                 57003 non-null  float64
 6   Proximity to Star              57055 non-null  float64
 7   Magnetic Field Strength        56942 non-null  object 
 8   Radiation Levels               56979 non-null  object 
 9   Atmospheric Composition Index  57058 non-null  float64
 10  Prediction                     56961 non-null  float64
dtypes: float64(9), object(2)
memory usage: 5.0+ MB


In [78]:
df.isnull().sum()

,0
Atmospheric Density,2984
Surface Temperature,3032
Gravity,2984
Water Content,3077
Mineral Abundance,2921
Orbital Period,2997
Proximity to Star,2945
Magnetic Field Strength,3058
Radiation Levels,3021
Atmospheric Composition Index,2942


In [79]:
df = df.fillna(df.mean(numeric_only=True))
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].fillna(df[col].mode()[0])
df.isnull().sum()


,0
Atmospheric Density,0
Surface Temperature,0
Gravity,0
Water Content,0
Mineral Abundance,0
Orbital Period,0
Proximity to Star,0
Magnetic Field Strength,0
Radiation Levels,0
Atmospheric Composition Index,0


In [80]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [81]:
df.columns = df.columns.str.normalize('NFKC').str.replace(r'\s+', ' ', regex=True).str.strip()


In [85]:
num_cols = ["Atmospheric Density", "Surface Temperature", "Gravity","Water Content", "Mineral Abundance", "Orbital Period","Proximity to Star", "Atmospheric Composition Index"]
cat_cols = ["Magnetic Field Strength", "Radiation Levels"]

In [86]:
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

In [89]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [90]:
X = df.drop("Prediction", axis=1)
y = df["Prediction"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42,stratify=y
)

In [91]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((48000, 46), (12000, 46), (48000,), (12000,))

In [92]:
def logistic_regression_model(X_train, X_test, y_train, y_test):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("--- Logistic Regression ---")
    evaluate_model(y_test, y_pred)


def svm_model(X_train, X_test, y_train, y_test):
    model = SVC(kernel="linear")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("--- SVM ---")
    evaluate_model(y_test, y_pred)

In [93]:
def evaluate_model(y_true, y_pred):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average="weighted"))
    print("Recall:", recall_score(y_true, y_pred, average="weighted"))
    print("F1 Score:", f1_score(y_true, y_pred, average="weighted"))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("--------------------------------------------------------\n")

In [ ]:
X = df.drop(columns=["Prediction"])
y = df["Prediction"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

logistic_regression_model(X_train, X_test, y_train, y_test)
svm_model(X_train, X_test, y_train, y_test)

--- Logistic Regression ---
Accuracy: 0.7230833333333333
Precision: 0.6988126265783244
Recall: 0.7230833333333333
F1 Score: 0.7098105283154146

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.81      0.82      1122
           1       0.80      0.89      0.84      1268
           2       0.72      0.74      0.73      1117
           3       0.70      0.74      0.72      1155
           4       0.71      0.75      0.73      1121
           5       0.06      0.01      0.02       562
           6       0.63      0.63      0.63      1058
           7       0.79      0.83      0.81      1125
           8       0.82      0.83      0.82      1236
           9       0.68      0.70      0.69      1100
          10       0.62      0.64      0.63      1136

    accuracy                           0.72     12000
   macro avg       0.67      0.69      0.68     12000
weighted avg       0.70      0.72      0.71     12000

Confusion Matrix:
 